# LCEL(LangChain Expression Language)
https://reference.langchain.com/python/langchain_core/runnables/

https://reference.langchain.com/python/langchain_core/runnables/?h=runnablelambd#langchain_core.runnables.base.RunnableLambda
  
- LCEL(LangChain Expression Language)은 LangChain에서 체인을 선언적으로 구성할 수 있게 해주는 도메인 특화 언어다.  
- `|` 연산자를 사용해 프롬프트, 모델, 파서 등을 파이프라인처럼 연결한다.

**주요 특징**

- **선언적 문법**: Unix 파이프처럼 `chain = prompt | model | parser` 형태로 직관적이다.  
- **모듈성·유연성**: 프롬프트, LLM, 파서, 검색기, 메모리 등 컴포넌트를 자유롭게 조합할 수 있다.  
- **동기/비동기 지원**: 단일 코드로 동기식·비동기식 실행을 모두 처리할 수 있다.  
- **병렬 처리 최적화**: 병렬 실행 가능한 단계는 자동으로 병렬화해 지연 시간을 줄인다.  
- **고급 기능 기본 제공**:  
  - 스트리밍 출력으로 응답 속도를 향상시킨다.  
  - 실패 시 재시도와 폴백 경로를 설정할 수 있다.  
  - 중간 결과에 접근해 디버깅이나 진행 상황 표시가 가능하다.

**LCEL의 주요 기능**

1. **스트리밍 지원**: 첫 토큰 도달 시간을 단축해 실시간성을 높인다.  
2. **비동기 지원**: asyncio 환경 등 다양한 실행 환경을 동일 코드로 지원한다.  
3. **병렬 실행 최적화**: 병렬화 가능한 단계는 자동으로 분리해 동시에 실행한다.  
4. **재시도·폴백 구성**: 오류 발생 시 지정 횟수만큼 재시도하거나 대체 경로를 실행한다.  
5. **중간 결과 접근**: 최종 출력 이전에 각 단계의 출력을 확인할 수 있다.

**기본 구성 요소**

- **Runnable**: LCEL의 모든 컴포넌트가 상속하는 기본 클래스다.  
- **Chain**: 여러 Runnable을 순차적으로 실행한다.  
- **RunnableMap**: 여러 Runnable을 병렬로 실행한다.  
- **RunnableSequence**: Runnable들의 시퀀스를 정의한다.  
- **RunnableLambda**: 파이썬 함수를 래핑해 Runnable로 만든다.

In [1]:
%pip install langchain langchain-openai -Uqqq

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv  # .env 파일의 환경변수 로드
import os                       # 환경변수 접근용

load_dotenv()                                                         # 현재 위치의 .env를 읽어와 환경변수로 등록
os.environ["OPENAI_API_KEY"] = os.getenv("openai_key")                # .env의 openai_key 값을 OPENAI_API_KEY로 등록
os.environ["LANGSMITH_TRACING"] = 'true'                              # LangSmith 트레이싱 활성화
os.environ["LANGSMITH_ENDPOINT"] = 'https://api.smith.langchain.com'  # LangSmith API 엔드포인트 설정
os.environ["LANGSMITH_PROJECT"] = 'skn23-langchain'                   # LangSmith 프로젝트명 설정
os.environ["LANGSMITH_API_KEY"] = os.getenv("langsmith_key")          # .env의 langsmith_key 값을 LANGSMITH_API_KEY로 등록

## RunnableLambda
일반 python 함수를 lcel체인에서 사용할수 있게 wrapping 처리하는 클래스

In [3]:
from langchain_core.runnables import RunnableLambda     # 입력을 받아 함수를 실행하는 Runnable

runnable = RunnableLambda(lambda x: len(x))             # 입력 x의 길이를 반환하는 Runnable 생성
runnable.invoke("플레이데이터 독산 skn-23 화이팅!")

21

In [4]:
# Batch :  여러 건의 입력을 일괄처리하는 메소드
runnable.batch(['플레이데이터', '독산', 'SKN-23', '화이팅!'])

[6, 2, 6, 4]

In [5]:
# 섭씨 입력값을 화씨로 변환하는 runnable 생성

def celsius_to_fahrenheit(celsius):
    """섭씨 온도를 화씨 온도로 변환하는 함수"""
    return celsius * 9 / 5 + 32     # 섭씨 -> 화씨 변환 공식

celsius_temps = [0, 25, 100, -10, 37]
runnable = RunnableLambda(celsius_to_fahrenheit)    # 함수를 Runnable로 래 핑
runnable.batch(celsius_temps)                       # 여러 입력을 한 번에 변환


[32.0, 77.0, 212.0, 14.0, 98.6]

In [6]:
import time

def gen(x):
    """입력 문자열을 한 글자(문자)씩 yield하는 제너레이터"""
    for y in x:                 # 입력을 순회(문자 단위)
        yield y                 # 한 글자씩 반환(스트리밍 단위)
        

runnable = RunnableLambda(gen)  # 문자열 제너레이터 함수를 Runnable로 래핑

for chunk in runnable.stream("안녕하세요? 안녕하세요! 안녕하세요 ~안녕하세요? 안녕하세요! 안녕하세요~ 안녕하세요? 안녕하세요! 안녕하세요~ 안녕하세요? 안녕하세요! 안녕하세요~안녕하세요? 안녕하세요! 안녕하세요~안녕하세요? 안녕하세요! 안녕하세요~"):
    print(chunk, end='', flush=True)    # chunk를 줄바꿈 없이 즉시 출력
    time.sleep(0.1)

안녕하세요? 안녕하세요! 안녕하세요 ~안녕하세요? 안녕하세요! 안녕하세요~ 안녕하세요? 안녕하세요! 안녕하세요~ 안녕하세요? 안녕하세요! 안녕하세요~안녕하세요? 안녕하세요! 안녕하세요~안녕하세요? 안녕하세요! 안녕하세요~

In [7]:
def gen(x):
    """iterable을 받아 원소를 하나씩 yield하는 제너레이터"""
    for y in x:                 # 입력을 순회(iterable)
        yield y                 # 원소를 하나씩 순회
        
gen10 = gen(range(10))          # 0~9까지 하나씩 꺼내는 제너레이터 생성
gen10                           # 객체 자체를 실행

<generator object gen at 0x000002C0F5778940>

In [8]:
for n in gen10: # gen10에서 값을 하나씩 꺼내며 순회(꺼낸 값은 다시 사용못함)
    print(n)

0
1
2
3
4
5
6
7
8
9


In [9]:
gen10 = gen(range(10))

In [10]:
next(gen10) # 제너레이터에서 다음 값을 1개 반환

0

## RunnableSequence
Runnable 객체를 순차 연결하는 Runnable 객체

In [11]:
from langchain_core.runnables import RunnableSequence    # Runnable들을 순서대로 연결하는 시퀀스다

runnable1 = RunnableLambda(lambda x: {'foo': x})        # 입력 x 를 {'foo': x} 형태로 변환
runnable2 = RunnableLambda(lambda x: [x] * 3)           # 입력 x를 리스트를 3번 반복 [x, x, x]

chain = RunnableSequence(runnable1, runnable2)
#chain = runnable1 | runnable2 위와 동일
chain.invoke(3)

[{'foo': 3}, {'foo': 3}, {'foo': 3}]

## RunnableParellel
여러 Runnable 객체를 인자로 받아, 병렬 처리 후 각각의 응답을 하나의 dict형태로 반환

In [12]:
from langchain_core.runnables import RunnableParallel   # 여러 Runnable들을 같은 입력으로 병렬 실행

runnable1 = RunnableLambda(lambda x: {'foo': x})        # 입력 x 를 {'foo': x} 형태로 변환
runnable2 = RunnableLambda(lambda x: [x] * 3)           # 입력 x를 리스트를 3번 반복 [x, x, x]

chain = RunnableParallel(r1 = runnable1, r2 = runnable2)  # r1, r2를 병렬로 실행해 결과를 dict로 묶음

chain.invoke(3)

{'r1': {'foo': 3}, 'r2': [3, 3, 3]}

사용자가 지정한 주제(topic)에 대해서 삼행시, 농담,시를 각각 생성하여 하나의 응답을 작성한다.

In [14]:
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableLambda

llm = init_chat_model("openai:gpt-4.1-mini")
output_parser = StrOutputParser()

acrostic_poem_prompt = PromptTemplate.from_template(
    "당신의 20년 경력의 n행시 고수입니다. 다음 주제로 n행시를 지어주세요. \n\n주제:{topic}"    
)
acrostic_poem_chain = acrostic_poem_prompt | output_parser | llm

In [ ]:
from langchain_core.prompts import PromptTemplate                           # 문자열 프롬프트 템플릿(변수 {topic} 등) 만들기
from langchain.chat_models import init_chat_model                           # OpenAI 등 Chat 모델 초기화 함수
from langchain_core.output_parsers import StrOutputParser                   # LLM 출력 -> 문자열로 뽑아주는 파서
from langchain_core.runnables import RunnableParallel, RunnableLambda       # 병렬 실행/사용자 함수 연결용 Runnable

llm = init_chat_model("openai:gpt-4.1-mini")                                # 사용할 LLM 모델 생성
output_parser = StrOutputParser()                                           # 모델 출력에서 최종 문자열만 추출하도록 파서 준비

# 1) n행시 프롬프트/체인
acrostic_poem_prompt = PromptTemplate.from_template(
    "당신의 20년 경력의 n행시 고수입니다. 다음 주제로 n행시를 지어주세요. \n\n주제:{topic}"
)
acrostic_poem_chain = acrostic_poem_prompt | llm | output_parser            # 프롬프트 -> LLM -> 문자열 파싱

# 2) 농담 프롬프트/체인
joke_poem_prompt = PromptTemplate.from_template(
    "당신의 20년 경력의 농담 고수입니다. 다음 주제로 허를 찌르는 농담을 하나 해주세요 \n\n주제:{topic}"
)
joke_chain = joke_poem_prompt | llm | output_parser                         # 프롬프트 -> LLM -> 문자열 파싱

# 3) 시 프롬프트/체인
apoem_prompt = PromptTemplate.from_template(
    "당신의 20년 경력의 시인입니다. 다음 주제로 감성적인 시를 지어주세요 \n\n주제:{topic}"
)
poem_chain = apoem_prompt | llm | output_parser                             # 프롬프트 -> LLM -> 문자열 파싱

# 4) 병렬 체인: 같은 입력(topic)을 3개의 체인에 동시에 넣고 결과를 딕셔너리로 모음
chain = RunnableParallel(
    acrotic_poem=acrostic_poem_chain,                                       # key: "acrotic_poem" (n행시 결과)
    joke=joke_chain,                                                        # key: "joke" (농담 결과)
    poem=poem_chain                                                         # key: "poem" (시 결과)
)

# 5) 병렬 실행 결과(딕셔너리)를 보기 좋게 하나의 문자열로 합치는 함수
def combine_result(input_dict: dict) -> str:
    """병렬 결과(n행시/농담/시)를 한 문자열로 합쳐 반환한다."""
    acrotic_poem = input_dict['acrotic_poem']                               # n행시 결과 꺼내기
    joke = input_dict['joke']                                               # 농담 결과 꺼내기
    poem = input_dict['poem']                                               # 시 결과 꺼내기
    return f""" 
n행시:
{acrotic_poem}

농담:
{joke}

시:
{poem}
"""
# 6) 체인 뒤에 RunnableLambda를 붙여서, 병렬 결과를 combine_result로 포맷팅하도록 연결
chain = chain | RunnableLambda(combine_result)



# 7) 실행: topic에 '아이스크림'을 넣으면 3개 결과를 병렬 생성 후 하나로 합쳐 출력
print(chain.invoke({'topic': '아이스크림'}))


 
n행시:
물론입니다! 주제 '아이스크림'으로 n행시 지어드리겠습니다.

아: 아이처럼 설레는 순간,  
이: 이 달콤함 한 입에 담아서,  
스: 스르르 녹아드는 행복을,  
크: 크고 작은 기쁨으로 나누어요,  
림: 림없이 퍼지는 여름의 추억처럼!  

즐거운 하루 되세요!

농담:
아이스크림 얘기하다가 갑자기 심각해지면 안 되죠! 

“아이스크림이 고민이 많다면? 그건 녹아 내리는 ‘내면의 갈등’ 때문일걸요!”

시:
아이스크림

여름 햇살 속 작은 위로  
손끝에 녹아 내리는 달콤함  
순간의 차가움에 기대어  
어린 날 꿈을 다시 본다  

입 안 가득 퍼지는 기억의 맛  
초콜릿, 바닐라, 딸기빛 향기  
무더운 시간 잠시 멈추고  
마음 한 켠에 작은 시원함을 남긴다  

아이스크림이 녹듯  
시간도 흘러가지만  
그 찰나의 환희는  
가슴 깊이 영원하리라



## RunnablePassThought
- 사용자의 입력값을 그대로 전달
- 입력 dict를 확장시켜준다.

In [ ]:
from langchain_core.runnables import RunnablePassthrough                    # 입력을 그대로 통과시키는 Runnable

llm = init_chat_model("openai:gpt-4.1-mini")                                # 사용할 LLM 모델 생성
output_parser = StrOutputParser()                                           # 모델 출력에서 최종 문자열만 추출하도록 파서 준비

# 1) n행시 프롬프트/체인
prompt = PromptTemplate.from_template(
    "당신의 창의적인 n행시 고수입니다. 다음 주제로 n행시를 지어주세요. \n\n주제:{topic}"
)

chain = ({'topic' : RunnablePassthrough()}  # 입력(문자열)을 그대로 받아 {'topic' : 입력}으로 매핑
        | prompt
        | llm 
        | output_parser)

chain.invoke("텀블러")


"물론입니다! 주제 '텀블러'로 n행시를 지어드릴게요.\n\n**텀**: 텀블러 안에 담긴 따뜻한 온기  \n**블**: 블링블링 빛나는 나만의 휴식처  \n**러**: 러닝 중에도 함께하는 소중한 친구  \n\n필요하시면 더 길거나 다른 스타일로도 만들어 드릴 수 있습니다!"

In [18]:
from langchain_core.runnables import RunnablePassthrough

prompt = PromptTemplate.from_template("""
당신은 창의적인 {n}행시 고수입니다. 다음주제로 {n}행시를 지어주세요.                                      

# 주제:
{topic}

# 출력형식 :
===== <주제> <n> 행시 =====
<n행시 작성>                                                                                                                                                                                                                                                                          
"""                                      
)
chain = (
    {'topic' : RunnablePassthrough()}
    | RunnablePassthrough.assign(       # 기존 dict를 확장해서 새 key를 추가
        n = lambda x : len(x['topic'])  # topic 길이로 n 계산
    )
    | prompt
    | llm
    | output_parser
)

chain.invoke('학원')

'===== 학원 2 행시 =====  \n학: 학교보다 더 깊이 배우고  \n원: 원대한 꿈을 키워 가는 곳'